In [ ]:
import matplotlib.pyplot as plt
import benchmark_visualization as bv
import numpy as np
import glob
import os
import json

HOME_DIR = os.environ['HOME']

SHORT_TIMELIMIT = 600
LONG_TIMELIMIT = 3600

runs = {}
class BenchmarkRun:
    HEADER = "algorithm,graph,timeout,seed,k,epsilon,num_threads,imbalance,totalPartitionTime,objective,km1,cut,failed"
    
    def __init__(self, content: str, timelimit: int = None):
        self.data = {}
        for i, key in enumerate(self.HEADER.split(',')):
            value = content.split(',')[i].strip()
            # Try to convert to int or float if possible
            try:
                if '.' in value:
                    value = float(value)
                else:
                    value = int(value)
            except ValueError:
                pass
            self.data[key] = value
        self.data['timelimit'] = timelimit

    def get(self, param):
        return self.data[param]

def parse_results_file(file_path: str):
    results_array = []
    timelimit = file_path.split('.')[-2]
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            results_array += [BenchmarkRun(content=line, timelimit=int(timelimit))]
    return results_array

def aggregate_runs(directory_path: str):
    runs = {}
    for file_path in glob.glob(directory_path + "/*.results"):
        file_name = file_path.split('/')[-1]
        instance_name = '.'.join(file_name.split('.')[:-2])
        if instance_name not in runs:
            runs[instance_name] = {"short": None, "long": None}

        run_results = parse_results_file(file_path)
        if len(run_results) > 1:
            min_km1=min([run.get('km1') for run in run_results if run.get('failed') == 'no'])
            runs[instance_name]["short"] = min_km1
        elif run_results:
            runs[instance_name]["long"] = run_results[0].get('km1')
        else:
            pass
            #print(f"Warning: No results in file {file_path}")

    return runs



def convert_instance_naming_scheme(instance_name: str) -> str:
    parts = instance_name.split('.')
    hgr_index = parts.index('hgr')
    base_name = '.'.join(parts[:hgr_index + 1])
    
    # Extract parameters from the rest
    k_value = None
    seed_value = None
    timelimit_value = None

    for part in parts[hgr_index + 1:]:
        if part.startswith('k'):
            k_value = part[1:]
        elif part.startswith('seed'):
            seed_value = part[4:]
        elif part.startswith('timelimit'):
            timelimit_value = part[9:]
    
    # Construct new name: base.seed.k.seed.timelimit
    new_name = f"{base_name}.{seed_value}.{k_value}.{seed_value}.{timelimit_value}"
    return new_name


def parse_end_result_history_file(file_path: str):
    # Read the last number from the file
    with open(file_path, 'r') as f:
        lines = f.readlines()
        if lines:
            last_line = lines[-1].strip()
            km1 = int(last_line.split(',')[-1].strip())
        else:
            km1 = None
    return km1

def parse_history_file(file_path: str):
    history = []
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue         
            if line.startswith('Starttime:'):
                timestamp = int(line.split(':')[1].strip())
                history.append((timestamp, None, None))
            else:
                parts = line.split(',')
                timestamp = int(parts[0].strip())
                mode = parts[1].strip()
                km1_value = int(parts[2].strip())
                history.append((timestamp, mode, km1_value)) 
    return history

def get_average_diff_from_matrix(matrix):
    np_matrix = np.array(matrix)
    mask = ~np.eye(np_matrix.shape[0], dtype=bool)
    off = np_matrix[mask]
    mean = (off.mean() if off.size else 0)
    return mean

def get_average_diff_from_matrices(matrices):
    averages = []
    for matrix in matrices:
        avg = get_average_diff_from_matrix(matrix)
        averages.append(avg)
    return averages

def get_max_diff_from_matrix(matrix):
    np_matrix = np.array(matrix)
    max_val = np_matrix.max()
    return max_val

def get_max_diff_from_matrices(matrices):
    max_values = []
    for matrix in matrices:
        max_val = get_max_diff_from_matrix(matrix)
        max_values.append(max_val)
    return max_values

def plot_combined_data(combined_data, title: str = "Combined History and Difference Matrices"):
    timestamps = [entry['timestamp'] for entry in combined_data]
    km1_values = [entry['km1'] for entry in combined_data]
    avg_diffs = [get_average_diff_from_matrix(entry['diff_matrix']) for entry in combined_data]
    max_diffs = [get_max_diff_from_matrix(entry['diff_matrix']) for entry in combined_data]

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

    # First subplot: KM1 values
    color = 'tab:blue'
    ax1.set_xlabel('Time (seconds)', fontsize=12)
    ax1.set_ylabel('KM1 Value', color=color, fontsize=12)
    ax1.plot(timestamps, km1_values, color=color, marker='o', label='KM1 Value')
    ax1.tick_params(axis='y', labelcolor=color)
    ax1.set_title('KM1 Value over Time', fontsize=14, fontweight='bold')
    ax1.grid(True, alpha=0.3)

    # Second subplot: Average and Max Differences
    ax2.set_xlabel('Time (seconds)', fontsize=12)
    ax2.set_ylabel('Difference Value', fontsize=12)
    ax2.plot(timestamps, avg_diffs, color='tab:red', marker='x', label='Average Difference')
    ax2.plot(timestamps, max_diffs, color='tab:orange', marker='s', label='Max Difference')
    ax2.legend(loc='upper right')
    ax2.set_title('Difference Metrics over Time', fontsize=14, fontweight='bold')
    ax2.grid(True, alpha=0.3)

    fig.suptitle(title, fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

def combine_history_and_diff(history_run, diff_run):
    combined_data = []
    index = 0
    start_time = None
    
    for step in history_run:
        timestamp, mode, km1_value = step
        if mode == None:
            start_time = timestamp
            continue
        if mode == 'Initial':
            # skip
            continue
        
        relative_time = (timestamp - start_time) / 1000.0  
        
        combined_data.append({
            'timestamp': relative_time,
            'mode': mode,
            'km1': km1_value,
            'diff_matrix': diff_run[index]
        })
        index += 1
    
    return combined_data

def aggregate_history_runs(directory_path: str):
    history_runs = {}
    for file_path in glob.glob(directory_path + "/*.csv"):
        file_name = os.path.basename(file_path)
        thread_id = file_name.split('.')[-3]
        instance_name = '.'.join(file_name.split('.')[:-3])
        instance_name = convert_instance_naming_scheme(instance_name)
        history = parse_history_file(file_path)
        
        # Last entry should be the final result (skip Starttime and Initial)
        km1 = None
        for timestamp, mode, km1_value in reversed(history):
            if mode is not None:
                km1 = km1_value
                break

        # Store only the history for the best run
        if instance_name not in history_runs:
            history_runs[instance_name] = {'thread_id': thread_id, 'km1': km1, 'history': history}
        else:
            if history_runs[instance_name]['km1'] is None:
                pass
            if km1 is not None and km1 < history_runs[instance_name]['km1']:
                history_runs[instance_name] = {'thread_id': thread_id, 'km1': km1, 'history': history}

    return history_runs

def get_diff_matrices_for_best_run(history_runs, diff_matrices_list, instance_name: str):
    if instance_name not in history_runs:
        return None

    best_thread_id = history_runs[instance_name]['thread_id']
    for entry in diff_matrices_list:
        if entry['thread_id'] == best_thread_id:
            return entry['matrices']
    return None


def aggregate_diff_runs(directory_path: str):
    diff_runs = {}
    for file_path in glob.glob(directory_path + "/*.csv"):
        matrices = bv.parse_diff_matrices(file_path)
        file_name = os.path.basename(file_path)
        thread_id = file_name.split('.')[-3]
        instance_name = '.'.join(file_name.split('.')[:-3])
        instance_name = convert_instance_naming_scheme(instance_name)

        if instance_name not in diff_runs:
            diff_runs[instance_name] = []    
        diff_runs[instance_name].append({
            'thread_id': thread_id,
            'matrices': matrices
        })

    return diff_runs

def plot_single_matrix(matrix, title: str = "Difference Matrix"):
    plt.figure(figsize=(10, 8))
    if not isinstance(matrix, np.ndarray):
        matrix = np.array(matrix)
    
    im = plt.imshow(matrix, cmap='viridis', aspect='auto', interpolation='nearest')
    
    plt.colorbar(im, label='Difference Value')
    
    # Set labels and title
    plt.title(title, fontsize=14, fontweight='bold')
    # plt.xlabel('Iteration', fontsize=12)
    # plt.ylabel('Iteration', fontsize=12)
    plt.grid(False)
    
    # Tight layout for better spacing
    plt.tight_layout()
    
    plt.show()



### Various different results analysis ###

In [ ]:
RESULT_FILES_DIR = f"{HOME_DIR}/Documents/experiment_results/2025-10-21_test_results/mt_kahypar_evo_results"
DIFF_FILES_DIR = f"{HOME_DIR}/Documents/experiment_results/evo_diff_results"
HISTORY_FILES_DIR = f"{HOME_DIR}/Documents/experiment_results/evo_results_results"

# Aggregate runs from result files
runs = aggregate_runs(RESULT_FILES_DIR)

diff_array = []
for instance_name, results in runs.items():
    short_km1 = results["short"]
    long_km1 = results["long"]
    if short_km1 is not None and long_km1 is not None:
        diff = long_km1 - short_km1
        diff_array.append(diff)

# How often is long run better than short runs (percentage)
if diff_array:
    better_count = sum(1 for d in diff_array if d < 0)
    percentage_better = (better_count / len(diff_array)) * 100
    print(f"Percentage of instances where long run is better than short runs: {percentage_better:.2f}%")
else:
    print("No valid differences to analyze.")

print ("diff_array:", diff_array)

# Analyze largest relative differences (in favor of long runs)
relative_diffs = []
for run, km1 in runs.items():
    short_km1 = km1["short"]
    long_km1 = km1["long"]
    if short_km1 is not None and long_km1 is not None:
        diff = long_km1 - short_km1
        relative_diff = diff / short_km1
        relative_diffs.append((run, relative_diff))
relative_diffs.sort(key=lambda x: x[1], reverse=True)
print("relative differences: ", relative_diffs[:10])


# Diff Matrices Analysis
diff_runs = aggregate_diff_runs(DIFF_FILES_DIR)

# Analyze diff matrices for worst long runs
history_runs = aggregate_history_runs(HISTORY_FILES_DIR)
for run, _ in relative_diffs[:10]:
    short_instance = f"{run}.{SHORT_TIMELIMIT}"
    long_instance = f"{run}.{LONG_TIMELIMIT}"   
    if short_instance in diff_runs and long_instance in diff_runs:
        
        short_matrices = diff_runs[short_instance]
        long_matrices = diff_runs[long_instance]

        best_short_matrices = get_diff_matrices_for_best_run(history_runs, short_matrices, short_instance)
        last_short_matrix = best_short_matrices[-1] if best_short_matrices else None
        last_long_matrix = long_matrices[0]['matrices'][-1]
        
        long_diff_run = long_matrices[0]['matrices']
        short_diff_run = best_short_matrices
        short_history_run = history_runs[short_instance]['history']
        long_history_run = history_runs[long_instance]['history']
        
        combined_short = combine_history_and_diff(short_history_run, short_diff_run)
        combined_long = combine_history_and_diff(long_history_run, long_diff_run)
        plot_combined_data(combined_short, title=f"Short Run Combined Data for {short_instance}")
        plot_combined_data(combined_long, title=f"Long Run Combined Data for {long_instance}")

        # if last_short_matrix and last_long_matrix:
        #     plot_single_matrix(last_short_matrix, title=f"Short Run Diff Matrix for {short_instance}")
        #     plot_single_matrix(last_long_matrix, title=f"Long Run Diff Matrix for {long_instance}")





Percentage of instances where long run is better than short runs: 80.23%
diff_array: [-9, -63, -25, -100, -86, -84, 34, -12, -7, -66, -37, 0, -79, -41, -21, -94, -34, -239, -17, -300, -320, -72, -551, -11, -93, 0, -9, -465, -87, -158, -297, -247, 0, 8, -10, 12, -6, 5, 17, 27, -42, -33, -256, -195, -22, -153, -161, -10, -117, -21, -24, 0, -16, -284, -224, 2, -83, -23, 17, -120, -3, -185, -65, -174, -25, 35, -13, -108, -94, -131, -16, 0, -3, -4, 104, -93, -292, -91, -147, 3, -57, -56, -19, -9, -30, 28]
relative differences:  [('cfd1.mtx.hgr.1.8.1', 0.017139090309822018), ('sat14_atco_enc2_opt1_05_21.cnf.hgr.1.8.1', 0.005836575875486381), ('sat14_dated-10-17-u.cnf.dual.hgr.1.8.1', 0.005419510236852669), ('laminar_duct3D.mtx.hgr.1.8.1', 0.0036968576709796672), ('mixtank_new.mtx.hgr.1.8.1', 0.003320683111954459), ('ISPD98_ibm09.hgr.1.32.1', 0.0027095951546063117), ('RFdevice.mtx.hgr.1.8.1', 0.002651692403681173), ('sat14_6s153.cnf.dual.hgr.1.32.1', 0.001802451333813987), ('vibrobox.mtx.hgr.